In [1]:
import os
import sys
import pathlib
import yaml
import numpy
import pandas
import xarray
import h5netcdf
from datetime import datetime
from datetime import date
from glob import glob
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
from cartopy import crs, feature
from matplotlib import pyplot as patches, colors

In [2]:
# file_locations = pathlib.Path(
#     '/scratch/rmueller/MIDOSS/Results/MOHID_results_locations_try3_10042022_18:58:46.yaml'
# )

file_locations = pathlib.Path(
    '/scratch/rmueller/MIDOSS/Results/try3/MOHID_results_locationsByMonth_try3_17062022_02:01:03.yaml'
)
graphics_dir = pathlib.Path(
    '/home/rmueller/projects/def-allen/rmueller/MIDOSS/analysis-rachael/notebooks/graphics/try3/'
)
   

In [3]:
with file_locations.open("rt") as f:
    run_paths = yaml.safe_load(f)


In [4]:
input_file = run_paths['all'][0]

In [5]:
len(run_paths['all'])

9833

In [6]:
# open NETCDF
with xarray.open_dataset(input_file, engine='h5netcdf') as ds:
    spill_start = ds.time[0]
    spill_end = ds.time[-1]
    # Select surface volume, concentration and dissolution
    vol3d=ds.OilWaterColumnOilVol_3D.isel({'grid_z': 39})
    conc3d=ds.OilConcentration_3D.isel({'grid_z': 39})
beach_threshold = 5e-3
BeachVolume=ds.Beaching_Volume.where(
        ds.Beaching_Volume>beach_threshold
    ) 

# open Lagrangian.dat
input_dir=os.path.dirname(os.path.abspath(input_file))
# identify oil type and run number from .nc file to query Lagrangian.dat file
nrun = input_file.split('/')[-1].split('-')[-1][:-3]
oil_tag = input_file.split('/')[-1].split('_')[1].split('-')[0]
Lagrangian_file = input_dir+f'/Lagrangian_{oil_tag}-{nrun}.dat'
 #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Grab spill information from Lagrangian file
with open(Lagrangian_file, 'r') as lagrangianFile:
    print(Lagrangian_file)
    for line in lagrangianFile:
        if 'POINT_VOLUME              :' in line: 
            # select spill volume quantity after ':'
            spillvolume = line.split(':')[-1].split('\n')[0]
        if 'POSITION_COORDINATES       :' in line:
            spill_location = line.split(':')[-1].split('\n')[0]
            print(spill_location)
        if 'POSITION_COORDINATES      :' in line:
            spill_location = line.split(':')[-1].split('\n')[0]
            print(spill_location)

/scratch/dlatorne/MIDOSS/runs/monte-carlo/1-200_near-BP_try3_2022-04-19T154852/results/1-200_near-BP_try3-0/Lagrangian_bunker-0.dat
 -122.7687378 48.85820007


In [7]:
regions = ['sog','cwchn','lummi','sji', 'ps','jdf']
sog = numpy.full(BeachVolume.shape, False)
cwchn = numpy.full(BeachVolume.shape, False)
lummi = numpy.full(BeachVolume.shape, False)
sji = numpy.full(BeachVolume.shape, False)
ps = numpy.full(BeachVolume.shape, False)
jdf = numpy.full(BeachVolume.shape, False)

# Strait of Georgia
sog[335:715,100:396] = True

# Cowichan
cwchn[340:500,150:295] = True

# Lummi
lummi[300:325,338:370] = True

# sji
sji[274:324,237:270]=True

#ps
ps[0:160,50:325]=True

#jdf
jdf[225:475,0:170]=True

masks = {}
for region in regions:
    masks[region] = numpy.ma.masked_array(globals()[region])

In [8]:
out={}
for region in regions:
    out[region]={}
    out[region]['lat'] = []
    out[region]['lon'] = []
    out[region]['vol']=[]
for region in regions:        
    volByRegion=numpy.nansum(numpy.asarray(BeachVolume)[masks[region]]) 
    print(volByRegion)
    if volByRegion > 0:
        print(region)
        out[region]['lat'].append(spill_location.split(' ')[2])
        out[region]['lon'].append(spill_location.split(' ')[1])
        out[region]['vol'].append(spillvolume)

1427.3339233034833
sog
0.0
0.0
0.0
0.0
0.0


In [9]:
len(run_paths['all'])

9833

## loop through all files and and find spill locations that affect region

In [10]:
regions

['sog', 'cwchn', 'lummi', 'sji', 'ps', 'jdf']

In [11]:
%%time
nruns = len(run_paths['all'])
run_list = run_paths['all']
beach_threshold = 5e-3
no_spills={}
spills={}
for region in regions:
    # store all spills for a particular region
    spills[region]={}
    spills[region]['lat'] = []
    spills[region]['lon'] = []
    spills[region]['vol']=[]
    spills[region]['spills_file'] = []
    # store all non-spills for a particular region
    no_spills[region]={}
    no_spills[region]['lat'] = []
    no_spills[region]['lon'] = []
    no_spills[region]['vol']=[]
    no_spills[region]['spills_file'] = []
for run in range(nruns):
    if run%500==0:
        print(f'Run {run} of {nruns}')
    input_file=run_list[run]
    input_dir=os.path.dirname(os.path.abspath(input_file))
    # identify oil type and run number from .nc file to query Lagrangian.dat file
    nrun = input_file.split('/')[-1].split('-')[-1][:-3]
    oil_tag = input_file.split('/')[-1].split('_')[1].split('-')[0]
    Lagrangian_file = input_dir+f'/Lagrangian_{oil_tag}-{nrun}.dat'
    if os.path.isfile(input_file):
        # Read in results from output netcdf
        try:
            with xarray.open_dataset(input_file, engine='h5netcdf') as ds:
                spill_start = ds.time[0]
                spill_end = ds.time[-1]
                # Select surface volume, concentration and dissolution
                vol3d=ds.OilWaterColumnOilVol_3D.isel({'grid_z': 39})
                conc3d=ds.OilConcentration_3D.isel({'grid_z': 39})

        except:
            print(f'missing {input_file}') 
            continue 
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Grab spill information from Lagrangian file
        with open(Lagrangian_file, 'r') as lagrangianFile:
            
            for line in lagrangianFile:
                if 'POINT_VOLUME              :' in line: 
                    # select spill volume quantity after ':'
                    spillvolume = line.split(':')[-1].split('\n')[0]
                if 'POSITION_COORDINATES       :' in line:
                    spill_location = line.split(':')[-1].split('\n')[0]
                if 'POSITION_COORDINATES      :' in line:
                    spill_location = line.split(':')[-1].split('\n')[0]
                   
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Store basics: Filename, lat, lon, spill volume
        #files.append(input_file)
        spill_volume=float(spillvolume)
        latitude=(float(spill_location.split(' ')[2]))
        longitude=(float(spill_location.split(' ')[1]))
        # ~~~~~~~~~~~~~~

        # Beaching 
        # ~~~~~~~~~~~~~~    
        # Save volume over threshold 
        BeachVolume=ds.Beaching_Volume.where(
            ds.Beaching_Volume>beach_threshold
        )
        
        for region in regions:
            volByRegion=numpy.nansum(numpy.asarray(BeachVolume)[masks[region]])           
            if volByRegion > 0:               
                spills[region]['lat'].append(latitude)
                spills[region]['lon'].append(longitude)
                spills[region]['vol'].append(float(spillvolume))
                spills[region]['spills_file'].append(input_file)
            else:
                no_spills[region]['lat'].append(latitude)
                no_spills[region]['lon'].append(longitude)
                no_spills[region]['vol'].append(float(spillvolume))
                no_spills[region]['spills_file'].append(input_file)
df={}
out_dir = pathlib.Path('/scratch/rmueller/MIDOSS/Results/try3/')
for region in regions:
    out_f = out_dir/f'{region}_SpillLocations.yaml'
    with open(out_f, "w") as out_file:
        yaml.safe_dump(spills[region], out_file)
        
    out_f = out_dir/f'{region}_NONESpillLocations.yaml'
    with open(out_f, "w") as out_file:
        yaml.safe_dump(no_spills[region], out_file)

Run 0 of 9833
Run 500 of 9833
Run 1000 of 9833
Run 1500 of 9833
Run 2000 of 9833
Run 2500 of 9833
Run 3000 of 9833
Run 3500 of 9833
Run 4000 of 9833
Run 4500 of 9833
Run 5000 of 9833
Run 5500 of 9833
Run 6000 of 9833
Run 6500 of 9833
Run 7000 of 9833
Run 7500 of 9833
Run 8000 of 9833
Run 8500 of 9833
Run 9000 of 9833
Run 9500 of 9833
CPU times: user 35min 32s, sys: 4min 54s, total: 40min 26s
Wall time: 1h 21min 49s
